# Sigmoid With Loss
/common/layers.py

In [2]:
class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], [] # 학습해야할 파라미터가 없음.
                                        # 그래서 그래디언트도 없음.
                                        # 다른 코드들이랑 형태 맞춰주려고 공리스트,,,
        self.loss = None
        self.y = None  # sigmoid의 출력
        self.t = None  # 정답 데이터

    def forward(self, x, t): # 순전파 하려면 시그모이드 층 통과해야...
                            # 그러려면, score 입력해야... x: score
        self.t = t
        self.y = 1 / (1 + np.exp(-x)) # x에 시그모이드 함수 적용시킨 것...

        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0] # t(라벨)의 길이가 배치사이즈

        dx = (self.y - self.t) * dout / batch_size # (y-t)에 흘러들어온 미분 곱하고 batch_size 로 나누기
        return dx

# Unigram Sampler
/ch04/negative_sampling_layer.py

In [ ]:
import collections
class UnigramSampler: # negative samling 구현하는 클래스
    def __init__(self, corpus, power, sample_size): # corpus: 말뭉치,
                                                    # power: 보정작업할 때 몇 승할 건지?
                                                    # 0에 가까울 수록 보정작업이 훨씬 드라마틱
                                                    # sample_size: negative sampling 몇 개 할건지 (총 sample_size + 1개만큼이 뽑힘. +1: 정답)
                                                    # 
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter() # 사전형식으로 value 추가하는 method
        for word_id in corpus: # 단어들의 idx (언제 처음 등장하는지)
            counts[word_id] += 1 # corpus 안에서 word_id가 총 몇 번 등장하는지
                                # counts = {word_id: 등장횟수}
                                # counts의 길이 len은 어휘의 개수

        vocab_size = len(counts) # vocab_size = counts의 길이 = 어휘의 개수
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size) # 어휘길이만큼 np.zeros
        for i in range(vocab_size):
            self.word_p[i] = counts[i] # 등장횟수를 채워줄 것

        self.word_p = np.power(self.word_p, power) # 보정작업, 원소 각각에 대해 power제곱
        self.word_p /= np.sum(self.word_p) # 각각의 원소에 대해 확률벡터 합으로 나눠주겠다. (normalize)

    def get_negative_sample(self, target): # target: contest에 의해 결정될 것.
        batch_size = target.shape[0]

        if not GPU: # gpu 사용 안할 때...
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32) # zero 행렬을 만들어서
                                                    # 1번째 context에 대한 negative sample, 2번째 ~~~, 3번째 ~~~...

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p) # random choice, replace = 린e: 비복원추출.
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)
                                               # random choice, replace = True: 중복해서 뽑아도 돼.

        return negative_sample

In [ ]:
# NegativeSamplingLoss
/ch04/negative_sampling_layer.py

In [ ]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] # 오답개수 +1(정답)
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh


In [ ]:
repeat 노드의 역전파 = sum 노드
sum 노드의 역전파 = repeat 노드
# sum 노드: 각 열 더하기
# repeat 노드: 각 열 반복해서 밑으로 계속 나열하기